In [ ]:
include("vector_spaces_library.jl")
include("mplstyle.jl");
import LinearAlgebra
LA = LinearAlgebra;

## Projection on a Subspace 


In [ ]:
v1 = [1.0,0.0,1.0]
v2 = [1.0,1.0,1.0]
v_basis = [v1,v2];

In [ ]:
x = rand(3)

In [ ]:
xstar = proj(P2,x,v_basis)

In [ ]:
xstar2, coeff = proj_normal(P2,x,v_basis)

In [ ]:
using3D()
pygui(true);
fig = figure("pyplot_surfaceplot",figsize=(15,10))
ax = fig.add_subplot(1,1,1,projection="3d")

ax[:scatter](v1[1], v1[2], v1[3], color="red")
ax[:plot]((v1[1],0), (v1[2],0),(v1[3],0), color="red", zorder = 3)
ax[:scatter](v2[1], v2[2], v2[3], color="red")
ax[:plot]((v2[1],0), (v2[2],0),(v2[3],0), color="red", zorder = 3)

ax[:scatter](x[1], x[2], x[3], color="green")
ax[:plot]((x[1],0), (x[2],0),(x[3],0), color="green", zorder = 3)

ax[:scatter](xstar[1], xstar[2], xstar[3], color="blue")
ax[:plot]((xstar[1],0), (xstar[2],0),(xstar[3],0), color="blue", zorder = 3)

# Exercise 11.3: Manufacturing Company

In [ ]:
Q = Matrix(LA.I, 10, 10)
A = vcat(ones(10)',[10 9 8 7 6 5 4 3 2 1])
b = [0;10^4];

In [ ]:
LQ = Val(:LQ) # Dispatch on the symbol LQ

function dot(::Val{:LQ}, x, y)
    return x' * Q * y
end

In [ ]:
# Take some x with Ax=b
start = A \ b

# Compute a basis for the null space of A
ns = LinearAlgebra.nullspace(A)
null_basis = [ns[:,i] for i=1:8]

# Project the start point on the subspace with Ax=0
p, coeff = proj_normal(LQ, start, null_basis)

# Move the start point but stay within the linear variety of Ax=b
minimizer = start - p

In [ ]:
# Norm of the result
dot(LQ, minimizer, minimizer)

In [ ]:
# Is the affine constraint fulfilled?
@assert sum((A * minimizer .- b) .< 0.01) == length(b)

# Exercise 11.4: Eigenfaces

In [ ]:
import Images
import Glob
import ImageView

# Force the pixel values on the interval [0,1]
function clamp_px(val)
    if val > 1.0
        return 1.0
    end
    if val < 0.0
        return 0.0
    end
    return val
end


In [ ]:
# Load the faces into the matrix F.
# Every column is one face
x = "datasets/att_faces/s1/1.pgm"
faces = map(x->Images.load(x), Glob.glob("*/*.pgm","datasets/att_faces"))
picsize = size(faces[1])
F = zeros(picsize[1]*picsize[2], length(faces))
for i = 1:length(faces)
   F[:,i] = reshape(map(Float64, faces[i]), size(F,1))
end

In [ ]:
faces

In [ ]:
# Subtract the average face to compute Fnormalized
avg_face = zeros(picsize[1]*picsize[2])
for i = 1:picsize[1]*picsize[2]
   avg_face[i] = sum(F[i,:]) / length(faces)
end
Fnormalized = F .- avg_face
Images.colorview(Images.Gray, reshape(avg_face, picsize))

In [ ]:
# Compute the vector space of the first N eigenfaces
# You don't have to understand this step to continue!
N = 200
AA = transpose(Fnormalized) * Fnormalized
EV = LinearAlgebra.eigvecs(AA)
eigenfaces = Fnormalized * EV[:,length(faces)-N+1:end]
Images.colorview(Images.Gray, reshape(eigenfaces[:,end-2], picsize));

In [ ]:
Images.colorview(Images.Gray, reshape(clamp_px.(eigenfaces[:,40] + avg_face), picsize))

In [ ]:
f = rand(1:length(faces)) # face index
face = Fnormalized[:,f]
# =========================================================================================
# Reconstruct the face at index f as a linar combination of the eigenfaces

facespace = Vector(undef, size(eigenfaces,2))

for i = 1:size(eigenfaces,2)
   facespace[i] = eigenfaces[:,i]
end

projection, coeff = proj_normal(P2, face, facespace)
# =========================================================================================

In [ ]:
[faces[f] Images.colorview(Images.Gray, reshape(projection + avg_face, picsize))]